In [7]:
from IPython.display import display, Markdown
display(Markdown('functionalities.md'))

Available functionalities  
===========================

Meshes
------

The simulation requires a mesh $M$ that can be defined either

- using the \ref cdmath structured mesh constructor Mesh $M(xmin,xmax,nx)$
- loading a med file generated by \ref salome platform Mesh $M$ (''  filename '')

All the boundary faces of the mesh sould be gathered into groups of FACES that will be used to set boundary conditions. These groups of faces can be set
- using the \ref cdmath face group generator \a M.setGroupAtPlan(value,direction,tolerance,groupName), where direction is an integer with value $0$ for the plane $x=value$, $1$ for the plane $y=value$ and $2$ for the plane $z=value$
- using \ref salome platform when generating the med file


\section Initialdata Initial data or power fields
The initial data and power fields can be defined on the simulation mesh using the Field class of \ref cdmath either
- starting from a null field using the command Field(fieldname,ON\_CELLS,numberOfComponents), then setting the values on cell $i$ for each component  $j$ with the command $F(i,j)=fij$
- loading a field stored in a med file using the command Field(filename,ON\_CELLS, filename)

The initial data should in general be a vector field containing parameters that are specific to each model
- SinglePhase : $(p, \vec{v}, T)$, ie pressure, velocity, temperature
- DriftModel : $(c_g, p, \vec{v}, T)$, ie phase 1 concentration, pressure, velocity, temperature
- IsothermalTwoFluid : $(\alpha_g, p,\vec{v}_g,\vec{v}_l)$, ie phase 1 void fraction, pressure, phase 1 velocity, phase 2 velocity
- FiveEqsTwoFluid : $(\alpha_g, p,\vec{v}_g,\vec{v}_l,T)$, ie phase 1 void fraction, pressure, phase 1 velocity, phase 2 velocity, common temperature
- TransportEquation : $h$ the fluid enthalpy 
- DiffusionEquation : $T$ the solid temperature

Whatever the model chosen, the initial data is set with the function ProblemCoreFlows::setInitialField (initialField), and the heat power field is set with the function ProblemCoreFlows::setHeatSource(heatPower) for a constant uniform value or the function ProblemCoreFlows::setHeatSourceField(heatPowerField) for a variable power field.

Models
------

The model can be set as

- SinglePhase::SinglePhase ( \ref fluidType, \ref pressureEstimate,spaceDimension) where \ref fluidType is either  \ref Liquid or \ref Steam, and \ref pressureEstimate is \ref around1bar300K or \ref around155bars600K
- DriftModel::DriftModel (\ref pressureEstimate,spaceDimension) where  pressureEstimate is \ref around1bar300K or \ref around155bars600K
- IsothermalTwoFluid::IsothermalTwoFluid (\ref pressureEstimate,spaceDimension) where  pressureEstimate is \ref around1bar300K or \ref around155bars600K
- FiveEqsTwoFluid::FiveEqsTwoFluid (\ref pressureEstimate,spaceDimension) where  pressureEstimate is \ref around1bar300K or \ref around155bars600K
- TransportEquation::TransportEquation (\ref fluidType, \ref pressureEstimate ,$\vec v$) where \ref fluidType is either  \ref Liquid or \ref Steam, \ref pressureEstimate is \ref around1bar300K or \ref around155bars600K and$\vec v$ is the velocity vector assumed constant and of type vector<double>
- DiffusionEquation::DiffusionEquation (spaceDim,$ \rho, c_p,\lambda$) where $\rho$ is the solid density, $c_p$ its specific heat and $\lambda$ its conductivity with default values set for Uranirum

Physical options
----------------

The following physical parameters have default value zero but can be set by the user:

- In order to set a constant heat source : ProblemCoreFlows::setHeatSource($ heatSource $) where heat source is a double
- In order to set the viscosities : ProblemFluid::setViscosity($ viscosite $ ) where viscosite is a $vector<double>$ of size nbPhases
- In order to set the conductivities : ProblemFluid::setConductivity ($ conductivite$ ) where conductivite is a $vector<double>$ of size nbPhases
- In order to set the gravity : ProblemFluid::setGravity($ gravite$ ) where gravite is a $vector<double>$ of size spaceDim
- In order to set the friction coefficients : ProblemFluid::setDragCoeffs($ dragCoeffs$ ) where dragCoeffs is a $vector<double>$ containing the friction coefficients between fluid and solid in a porous approach for SinglePhase (one coefficient) and DriftModel (two coefficients), or interfacial friction between the two phases for IsothermalTwoFluid (one coefficient) and FiveEqsTwoFluid (one coefficient)
- In order to set the heat transfert coefficient between fluid and solid in a porous flow ProblemCoreFlows::setHeatTransfertCoeff (heatCoeff)

Numerical options
-----------------

- The nonlinear formulation can be \ref Roe, \ref VFRoe or \ref VFFC and is set by the method ProblemFluid::setNonLinearFormulation(\ref NonLinearFormulation)
- The time discretisation can be \ref Explicit or \ref Implicit, and the space discretisation can be \ref upwind, \ref lowMach, \ref pressureCorrection, \ref staggered or \ref centered. The space and time discretisations can be set using the command ProblemCoreFlows::setNumericalScheme( \ref upwind ,  \ref Explicit ); or ProblemCoreFlows::setNumericalScheme( \ref centered ,  \ref Implicit); for instance. 
- Advanced options can be set to enforce a well balanced scheme with ProblemFluid::setWellBalancedCorrection or an entropic scheme  with ProblemFluid::setEntropicCorrection.
- In order to set the cfl number: ProblemCoreFlows::setCFL($ cfl $ );
- In order to set the tolerance number  : ProblemCoreFlows::setPrecision($ precision $ );
- In order to set the maximum number of time steps : ProblemCoreFlows::setMaxNbOfTimeStep($ MaxNbOfTimeStep$ );
- In order to set the maximum time of the computation : ProblemCoreFlows::setTimeMax($ maxTime$);
- In order to set the linear solver and preconditioner : ProblemCoreFlows::setLinearSolver(\ref GMRES, \ref ILU ).
The usual linear solvers are \ref GMRES and \ref BICGSTAB, and the usual "preconditionners" are \ref ILU and \ref LU.


Boundary conditions
-------------------

For the numerical treatment of the boundaries, it is important that each boundary (Group in Salome vocabulary) be assigned a \ref BoundaryType.

Boundary types can be 

- \ref Wall
      + simply set with the function SinglePhase::setWallBoundaryCondition, DriftModel::setWallBoundaryCondition, IsothermalTwoFluid::setWallBoundaryCondition, FiveEqsTwoFluid::setWallBoundaryCondition
      + possible for all fluid models : SinglePhase, DriftModel, IsothermalTwoFluid, FiveEqsTwoFluid
- \ref Neumann
      + simply set with the function ProblemCoreFlows::setNeumannBoundaryCondition
      + possible for all models
- \ref Inlet
      + simply set with the functions SinglePhase::setInletBoundaryCondition, SinglePhase::setInletBoundaryCondition, DriftModel::setInletBoundaryCondition, IsothermalTwoFluid::setInletBoundaryCondition, FiveEqsTwoFluid::setInletBoundaryCondition or TransportEquation::setInletBoundaryCondition
      + possible for TransportEquation, SinglePhase, DriftModel, IsothermalTwoFluid, FiveEqsTwoFluid
- \ref Outlet
      + simply set with the command ProblemFluid::setOutletBoundaryCondition
      + possible for all fluid models : SinglePhase, DriftModel, IsothermalTwoFluid, FiveEqsTwoFluid
- \ref Dirichlet
      + simply set with the command DiffusionEquation::setDirichletBoundaryCondition
      + possible for DiffusionEquation

The former way to set the boundary conditions was to create a C++ map $map<string, LimitField> boundaryFields$ that associates the boundary name to a field. Once the map is filled with the appropriate content, the boundary conditions are set with the command ProblemCoreFlows::setBoundaryFields(boundaryFields).
The LimitField structure is specific to each model and each type of boundary condition.
- for a Neumann boundary condition :  LimitField limitNeumann; limitNeumann.bcType=Neumann;

- for an Inlet boundary condition (not possible for DiffusionEquation): LimitField limitInlet; limitInlet.bcType=Inlet; then
      + For TransportEquation: 
		- limitInlet.h=1.3e6
      + For SinglePhase (3D case):
	  - limitInlet.T = 600;	
	  - limitInlet.v\_x = $vector<double>(1,0)$;
	  - limitInlet.v\_y = $vector<double>(1,0)$;
	  - limitInlet.v\_z = $vector<double>(1,0)$; 

     + For DriftModel (3D case):
	  - limitInlet.c=0;	
	  - limitInlet.T = 600;	
	  - limitInlet.v\_x = $vector<double>(1,0)$;
	  - limitInlet.v\_y = $vector<double>(1,0)$;
	  - limitInlet.v\_z = $vector<double>(1,0)$;

     + For IsothermalTwoFluid Model (3D case):	
	  - limitInlet.alpha=0;	
	  - limitInlet.v\_x = $vector<double>(2,0)$;
	  - limitInlet.v\_y = $vector<double>(2,0)$;	
	  - limitInlet.v\_z = $vector<double>(2,0)$;

     + For FiveEqsTwoFluid (3D case): 		
	  - limitInlet.alpha=0;	
	  - limitInlet.T = 600;	
	  - limitInlet.v\_x = $vector<double>(2,0)$;	
	  - limitInlet.v\_y = $vector<double>(2,0)$;	
	  - limitInlet.v\_z = $vector<double>(2,0)$; 

     + then boundaryFields['' \a Inlet'']= limitInlet;	

- for an Outlet boundary condition (for fluid models): LimitField limitOutlet; limitOutlet.bcType=Outlet; then
      + For all fluid models (SinglePhase, DriftModel, IsothermalTwoFluid, FiveEqsTwoFluid: 		
	  - limitOutlet.p = 155e5;
      + then boundaryFields[''\a outlet'']= limitOutlet;
      
- for a Wall with friction and/or heat conduction : LimitField limitWall; limitWall.bcType=Wall; then
      + For SinglePhase (3D case): 
	  - limitWall.T = 600;	
	  - limitWall.v\_x = $vector<double>(1,0)$;	
	  - limitWall.v\_y = $vector<double>(1,0)$; 	
	  - limitWall.v\_z = $vector<double>(1,0)$;
      + For DriftModel (3D case):		
	  - limitWall.T = 600;	
	  - limitWall.v\_x = $vector<double>(1,0)$;	
	  - limitWall.v\_y = $vector<double>(1,0)$;	
	  - limitWall.v\_z = $vector<double>(1,0)$;
      + For IsothermalTwoFluid Model (3D case):	
	  - limitWall.v\_x = $vector<double>(2,0)$;
	  - limitWall.v\_y = $vector<double>(2,0)$;
	  - limitWall.v\_z = $vector<double>(2,0)$;
      + For FiveEqsTwoFluid (3D case): 		
	  - limitWall.T = 600;
	  - limitWall.v\_x = $vector<double>(2,0)$;	
	  - limitWall.v\_y = $vector<double>(2,0)$;	
	  - limitWall.v\_z = $vector<double>(2,0)$;	
      + Then boundaryFields['' \a Wall '']= limitWall;	

- for a Dirichlet boundary condition (only for DiffusionEquation) : LimitField limitDirichlet; limitDirichlet.bcType=Dirichlet; then
  	- limitDirichlet.T = 600; 
	- boundaryFields['' \a Dirichlet'']= limitDirichlet; 


